In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import googlemaps

In [3]:
routes_csv = pd.read_csv('../data/routesDist.csv')

In [12]:
usSrcRoutes = routes_csv[routes_csv['SRCCTRY'] == 'United States']
print(len(usSrcRoutes))

usDestinRoutes = routes_csv[routes_csv['DESTINCTRY'] == 'United States']
print(len(usDestinRoutes))

13021
13016


In [34]:
mergeCols = ['AIRLINE_ID', 'SRC_AIRPT_ID', 'DESTIN_AIRPT_ID']

usRoutes = usSrcRoutes.merge(usDestinRoutes, how='outer', on=mergeCols)
len(usRoutes)

15519

In [32]:
usRoutesTest = pd.concat([usSrcRoutes,usDestinRoutes])
usRoutesTest = usRoutesTest.drop_duplicates()
len(usRoutesTest)

15519

In [60]:
#re-using variable above that was used to show concat and outer join provide similar data
usRoutes = usRoutesTest[['SRC_AIRPT_ID', 'SRCNAME', 'DESTIN_AIRPT_ID', 'DESTINNAME']]

In [84]:
def distCalc(x, y):
    file = open("../dataEngineering/gMapsAPIKey.txt")
    gMapsKey = file.read()
    file.close()

    gmaps = googlemaps.Client(key=gMapsKey)

    distance = gmaps.distance_matrix(x, y)['rows'][0]['elements'][0]

    try:
        meters = distance['distance']['value']
    except:
        meters = r'\N'

    try:
        seconds = distance['duration']['value']
    except:
        seconds = r'\N'

    return(meters,seconds)

In [66]:
usRoutes = usRoutes.reset_index()
usRoutes = usRoutes.rename(columns={'index': 'og_index'})

In [99]:
m = []
s = []

for i in range(len(usRoutes)):
    mVal, sVal = distCalc(usRoutes['SRCNAME'][i], usRoutes['DESTINNAME'][i])
    m.append(mVal)
    s.append(sVal)
    time.sleep(1)
    

KeyboardInterrupt: 

In [96]:
s

['\\N',
 '\\N',
 13361,
 11518,
 10612,
 8141,
 14302,
 13397,
 10653,
 11653,
 8409,
 14313,
 6162,
 6114,
 27636,
 4950,
 15644,
 14485,
 19301,
 15465,
 13585,
 18545,
 25398,
 '\\N',
 '\\N',
 13347,
 '\\N',
 '\\N',
 '\\N',
 10953,
 '\\N',
 '\\N',
 13743,
 11381,
 7025,
 7700,
 '\\N',
 12198,
 27433,
 4895,
 18650,
 10911,
 7533,
 18976,
 '\\N',
 23582,
 11086,
 14866,
 15590,
 14345,
 '\\N',
 '\\N',
 12352,
 19213,
 23263,
 23938,
 18226,
 10830,
 14413,
 14840,
 24977,
 13544,
 7917,
 12202,
 12271,
 23966,
 15049,
 15331,
 11331,
 15343,
 15179,
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 146894,
 80221,
 111846,
 121010,
 50671,
 54989,
 18421,
 55536,
 95989,
 114665,
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 '\\N',
 27341,
 '\\N',
 '\\N',
 '\\N',
 14049,
 '\\N',
 '\\N']